# Pork Chop Background Removal with rembg

This notebook processes pork chop images using the rembg library (U-2-Net model) to remove backgrounds while preserving edges perfectly.

**🚀 Open in Google Colab:**
1. Commit and push this notebook to GitHub
2. Go to https://colab.research.google.com
3. Click "File → Open notebook → GitHub tab"
4. Enter your repo: `your-username/MSL-tender`
5. Select `scripts/process_chops_colab.ipynb`

**OR** use direct link:
https://colab.research.google.com/github/YOUR-USERNAME/MSL-tender/blob/main/scripts/process_chops_colab.ipynb

**Steps:**
1. Install dependencies (GPU-accelerated)
2. Configure credentials (copy-paste from .env)
3. Process test batch (5 images)
4. Verify results at your dashboard
5. Process all remaining images

**⚡ Enable GPU:** Runtime → Change runtime type → GPU → Save

**No manual annotation required** - U-2-Net is pre-trained for salient object detection.

In [ ]:
!pip install rembg[gpu] pillow boto3 supabase requests tqdm

# Verify GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

## 2. Configure Credentials

**Copy-paste your credentials from .env file below:**

Find these in your local `.env` file (or `.env.local`):
- `SUPABASE_URL`
- `SUPABASE_SERVICE_KEY` (service_role key, not anon key)
- `R2_ACCOUNT_ID`
- `R2_ACCESS_KEY_ID`
- `R2_SECRET_ACCESS_KEY`
- `R2_BUCKET_NAME`
- `NEXT_PUBLIC_R2_PUBLIC_URL`

In [ ]:
import os

# PASTE YOUR CREDENTIALS HERE (replace the placeholder values)
credentials = """
SUPABASE_URL=https://your-project.supabase.co
SUPABASE_SERVICE_KEY=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9...
R2_ACCOUNT_ID=abc123def456
R2_ACCESS_KEY_ID=your_access_key_id
R2_SECRET_ACCESS_KEY=your_secret_access_key
R2_BUCKET_NAME=your-bucket-name
NEXT_PUBLIC_R2_PUBLIC_URL=https://pub-123abc.r2.dev
"""

# Parse and set environment variables
for line in credentials.strip().split('\n'):
    if '=' in line and not line.startswith('#'):
        key, value = line.split('=', 1)
        key = key.strip()
        value = value.strip()
        
        # Map NEXT_PUBLIC_R2_PUBLIC_URL to R2_PUBLIC_URL
        if key == 'NEXT_PUBLIC_R2_PUBLIC_URL':
            os.environ['R2_PUBLIC_URL'] = value
        else:
            os.environ[key] = value

# Verify all required credentials are set
required = [
    'SUPABASE_URL',
    'SUPABASE_SERVICE_KEY',
    'R2_ACCOUNT_ID',
    'R2_ACCESS_KEY_ID',
    'R2_SECRET_ACCESS_KEY',
    'R2_BUCKET_NAME',
    'R2_PUBLIC_URL'
]

missing = [key for key in required if not os.environ.get(key) or os.environ[key].startswith('your')]

if missing:
    print(f"❌ Missing credentials: {', '.join(missing)}")
    print("\nPlease update the credentials variable above with your actual values.")
else:
    print("✓ All credentials configured successfully!")
    print(f"  Supabase: {os.environ['SUPABASE_URL']}")
    print(f"  R2 Bucket: {os.environ['R2_BUCKET_NAME']}")
    print(f"  R2 Public URL: {os.environ['R2_PUBLIC_URL']}")

## 3. Initialize Clients

In [ ]:
from supabase import create_client
import boto3
from botocore.config import Config
from rembg import remove
from PIL import Image
import io
import requests
from tqdm.auto import tqdm
from urllib.parse import urlparse

# Initialize Supabase client
supabase = create_client(
    os.environ['SUPABASE_URL'],
    os.environ['SUPABASE_SERVICE_KEY']
)

# Initialize R2 client (S3-compatible)
r2 = boto3.client(
    's3',
    endpoint_url=f"https://{os.environ['R2_ACCOUNT_ID']}.r2.cloudflarestorage.com",
    aws_access_key_id=os.environ['R2_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['R2_SECRET_ACCESS_KEY'],
    config=Config(signature_version='s3v4'),
    region_name='auto'
)

print("✓ Clients initialized")

## 4. Helper Functions

In [ ]:
def extract_study_number_from_url(url: str) -> str:
    """Extract study number from image URL path."""
    parsed = urlparse(url)
    path_parts = parsed.path.strip('/').split('/')
    
    # Find 'original' in path and take next segment
    try:
        original_idx = path_parts.index('original')
        if original_idx + 1 < len(path_parts):
            return path_parts[original_idx + 1]
    except (ValueError, IndexError):
        pass
    
    # Fallback: return first non-empty segment
    for part in path_parts:
        if part and part != 'photos' and part != 'original':
            return part
    
    return 'unknown'


def download_image(url: str) -> Image.Image:
    """Download image from URL."""
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    return Image.open(io.BytesIO(response.content))


def process_image_with_rembg(image: Image.Image) -> tuple[Image.Image, dict]:
    """
    Remove background using rembg and return cropped image with metadata.
    
    Returns:
        (processed_image, metadata_dict)
    """
    # Convert to RGB if needed
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Remove background (returns RGBA with transparent background)
    output = remove(image)
    
    # Get bounding box of non-transparent pixels
    bbox = output.getbbox()
    
    if bbox is None:
        # No foreground detected
        return None, {'confidence': 0.0, 'error': 'No foreground detected'}
    
    x1, y1, x2, y2 = bbox
    
    # Add 2% margin
    width = x2 - x1
    height = y2 - y1
    margin_x = int(width * 0.02)
    margin_y = int(height * 0.02)
    
    x1 = max(0, x1 - margin_x)
    y1 = max(0, y1 - margin_y)
    x2 = min(output.width, x2 + margin_x)
    y2 = min(output.height, y2 + margin_y)
    
    # Crop to bounding box
    cropped = output.crop((x1, y1, x2, y2))
    
    # Replace transparent background with white
    final = Image.new('RGB', cropped.size, (255, 255, 255))
    final.paste(cropped, mask=cropped.split()[3])  # Use alpha channel as mask
    
    # Calculate confidence based on foreground area
    total_pixels = output.width * output.height
    foreground_pixels = (x2 - x1) * (y2 - y1)
    confidence = min(0.99, foreground_pixels / total_pixels)
    
    metadata = {
        'crop_x1': x1,
        'crop_y1': y1,
        'crop_x2': x2,
        'crop_y2': y2,
        'confidence': confidence
    }
    
    return final, metadata


def upload_to_r2(image: Image.Image, study_number: str, filename: str) -> str:
    """Upload processed image to R2 and return public URL."""
    # Convert image to JPEG bytes
    buffer = io.BytesIO()
    image.save(buffer, format='JPEG', quality=95, optimize=True)
    buffer.seek(0)
    
    # Upload to R2
    key = f"processed/{study_number}/{filename}"
    r2.put_object(
        Bucket=os.environ['R2_BUCKET_NAME'],
        Key=key,
        Body=buffer.getvalue(),
        ContentType='image/jpeg',
        CacheControl='public, max-age=31536000, immutable'
    )
    
    # Return public URL
    return f"{os.environ['R2_PUBLIC_URL']}/{key}"


def update_database(image_id: int, processed_url: str, metadata: dict):
    """Update database with processed image information."""
    supabase.table('sample_images').update({
        'processed_image_url': processed_url,
        'crop_x1': metadata['crop_x1'],
        'crop_y1': metadata['crop_y1'],
        'crop_x2': metadata['crop_x2'],
        'crop_y2': metadata['crop_y2'],
        'crop_confidence': metadata['confidence'],
        'crop_processed': True,
        'processed_at': 'now()'
    }).eq('id', image_id).execute()

print("✓ Helper functions defined")

## 5. Fetch Unprocessed Images

Get all images from database that haven't been processed yet.

In [ ]:
# Fetch unprocessed images
response = supabase.table('sample_images') \
    .select('id, image_url') \
    .is_('crop_processed', 'null') \
    .not_.is_('image_url', 'null') \
    .execute()

images_to_process = response.data
print(f"Found {len(images_to_process)} unprocessed images")

# Show first 5 examples
for img in images_to_process[:5]:
    print(f"  ID {img['id']}: {img['image_url'][:80]}...")

## 6. Process Images (Test Mode)

Process first 5 images as a test to verify everything works.

In [ ]:
# Test with first 5 images
test_images = images_to_process[:5]

results = {
    'success': 0,
    'failed': 0,
    'errors': []
}

for img_data in tqdm(test_images, desc="Processing test images"):
    try:
        # Download original image
        original = download_image(img_data['image_url'])
        
        # Process with rembg
        processed, metadata = process_image_with_rembg(original)
        
        if processed is None:
            results['failed'] += 1
            results['errors'].append({
                'id': img_data['id'],
                'error': metadata.get('error', 'Unknown error')
            })
            continue
        
        # Extract study number and filename
        study_number = extract_study_number_from_url(img_data['image_url'])
        filename = img_data['image_url'].split('/')[-1]
        
        # Upload to R2
        processed_url = upload_to_r2(processed, study_number, filename)
        
        # Update database
        update_database(img_data['id'], processed_url, metadata)
        
        results['success'] += 1
        
    except Exception as e:
        results['failed'] += 1
        results['errors'].append({
            'id': img_data['id'],
            'error': str(e)
        })

print(f"\n✓ Test complete: {results['success']} succeeded, {results['failed']} failed")
if results['errors']:
    print("\nErrors:")
    for err in results['errors']:
        print(f"  ID {err['id']}: {err['error']}")

## 7. View Test Results

Check the processed images at: **https://your-domain.vercel.app/admin/crop-test**

Verify:
- ✅ Blue background completely removed
- ✅ Chop edges preserved perfectly
- ✅ No blue "mohawk" artifacts
- ✅ Paper tags excluded
- ✅ Fat/meat coloring intact

## 8. Process All Remaining Images

⚠️ **Run this cell only after verifying test results look good!**

This will process ALL remaining unprocessed images (~1485 images).

In [ ]:
# Fetch all unprocessed images again (in case test images are now processed)
response = supabase.table('sample_images') \
    .select('id, image_url') \
    .is_('crop_processed', 'null') \
    .not_.is_('image_url', 'null') \
    .execute()

all_images = response.data
print(f"Processing {len(all_images)} images...\n")

results = {
    'success': 0,
    'failed': 0,
    'errors': []
}

# Process with progress bar
for img_data in tqdm(all_images, desc="Processing all images"):
    try:
        # Download original image
        original = download_image(img_data['image_url'])
        
        # Process with rembg
        processed, metadata = process_image_with_rembg(original)
        
        if processed is None:
            results['failed'] += 1
            results['errors'].append({
                'id': img_data['id'],
                'error': metadata.get('error', 'Unknown error')
            })
            continue
        
        # Extract study number and filename
        study_number = extract_study_number_from_url(img_data['image_url'])
        filename = img_data['image_url'].split('/')[-1]
        
        # Upload to R2
        processed_url = upload_to_r2(processed, study_number, filename)
        
        # Update database
        update_database(img_data['id'], processed_url, metadata)
        
        results['success'] += 1
        
    except Exception as e:
        results['failed'] += 1
        results['errors'].append({
            'id': img_data['id'],
            'error': str(e)
        })
    
    # Print progress every 100 images
    if (results['success'] + results['failed']) % 100 == 0:
        print(f"Progress: {results['success']} succeeded, {results['failed']} failed")

print(f"\n" + "="*50)
print(f"✓ COMPLETE: {results['success']} succeeded, {results['failed']} failed")
print("="*50)

if results['errors']:
    print(f"\n{len(results['errors'])} errors occurred:")
    for err in results['errors'][:10]:  # Show first 10 errors
        print(f"  ID {err['id']}: {err['error']}")
    if len(results['errors']) > 10:
        print(f"  ... and {len(results['errors']) - 10} more")

## 9. Summary Statistics

In [ ]:
# Get final statistics
stats = supabase.table('sample_images') \
    .select('crop_processed', count='exact') \
    .execute()

processed_count = supabase.table('sample_images') \
    .select('id', count='exact') \
    .eq('crop_processed', True) \
    .execute()

total = stats.count
processed = processed_count.count
remaining = total - processed

print(f"\nFinal Statistics:")
print(f"  Total images: {total}")
print(f"  Processed: {processed} ({100*processed/total:.1f}%)")
print(f"  Remaining: {remaining}")
print(f"\nView results at: https://your-domain.vercel.app/admin/crop-test")